In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from churn_eda import clean_data
from onehotencoding import set_ohe
from random_forest import rand_forest

In [ ]:
df_train = pd.read_csv('data/churn_train.csv')
df_test = pd.read_csv('data/churn_train.csv')
df_train = clean_data(df_train)
df_test = clean_data(df_test)
to_one_hot_encode = ['city', 'phone']
for each_hot in to_one_hot_encode:
    df_train = set_ohe(df_train, each_hot)
    df_test = set_ohe(df_test, each_hot)
    
y_train = df_train.pop('churn').values
y_test = df_test.pop('churn').values
X_train = df_train.values
X_test = df_test.values

## Voting Classifier ##

In [ ]:
log_clf = LogisticRegression(solver='liblinear')
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
gb_clf = GradientBoostingClassifier(learning_rate=0.1, 
                                   n_estimators=100, 
                                   min_samples_leaf=10,
                                  subsample=0.5)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('bag', bag_clf), ('gb', gb_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)

accuracy = voting_clf.score(X_test, y_test)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)

scorers = ['accuracy', 'precision', 'recall']
scores = [accuracy, precision, recall]

for each_scorer, each_score in zip(scorers, scores):
    print(f'{each_scorer}: {each_score:2.2f}')

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
def plot_roc_curve(ax, fpr, tpr, label=None):
    ax.plot(fpr, tpr, linewidth=2, label=label)
    ax.plot([0,1], [0,1], 'k--')
    ax.set_title(f'{label} ROC Curve')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate (Recall)')
fig, ax = plt.subplots()

plot_roc_curve(ax, fpr, tpr, 'Voting Classifier')

In [ ]:
print(f'ROC AOC: {metrics.roc_auc_score(y_test, y_pred):2.2f}')